# KGTK Browser Cache Setup

This note book will create the SQLite DB Cache and the required indices for KGTK Browser.

The required input parameters are:
- input_path: Path where the following files should be present
    1. labels.en.tsv.gz
    2. aliases.en.tsv.gz
    3. descriptions.en.tsv.gz
    4. claims.tsv.gz
    5. metadata.property.datatypes.tsv.gz
    6. qualifiers.tsv.gz
    7. metadata.pagerank.undirected.tsv.gz
    8. derived.isastar.tsv.gz

The files `metadata.pagerank.undirected.tsv.gz` and `derived.isastar.tsv.gz` are created by running [this](https://github.com/usc-isi-i2/kgtk/blob/dev/use-cases/Wikidata%20Useful%20Files.ipynb) notebook
- output_path: Output path 
- project_name: folder inside the `output_path` where the required files and cache will be created
- es_host: ES host, default `localhost`
- es_port: ES port, default `9200`
- es_index: name of the desired elasticsearch index, default `wikidata-dwd-kgtk-search-01`
- create_db: variable to control creation of sqlite database cache. `yes|no`, default `yes`
- create_es: variable to control creation of the ES index. `yes|no`, defaut `no`
**Cache file location:** `<output_path>/<project_name>/temp.<project_name>/wikidata.sqlite3.db`

In [1]:
import os
import pandas as pd
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
import kgtk.kypher.api as kapi

In [2]:
input_path = "/Users/amandeep/Github/kgtk-notebooks/datasets/arnold"
output_path = "/Volumes/saggu-ssd/wikidata-dwd-v2"

project_name = "kgtk-search-arnold"

files = 'label,pagerank_undirected,alias,description,claims,datatypes,qualifiers'

es_host = "http://localhost"
es_port = 9200
es_index = "wikidata-dwd-kgtk-search-01"

create_db = 'yes'
create_es = 'no'

In [3]:
files = files.split(',')

create_sqlite_cache = create_db.lower() == 'yes'
create_es_index = create_es.lower() == 'yes'

In [4]:
ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)

User home: /Users/amandeep
Current dir: /Users/amandeep/Github/kgtk-browser
KGTK dir: /Users/amandeep/Github
Use-cases dir: /Users/amandeep/Github/use-cases


In [5]:
ck.print_env_variables()

EXAMPLES_DIR: /Users/amandeep/Github/examples
GRAPH: /Users/amandeep/Github/kgtk-notebooks/datasets/arnold
USE_CASES_DIR: /Users/amandeep/Github/use-cases
STORE: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search-arnold/temp.kgtk-search-arnold/wikidata.sqlite3.db
KGTK_OPTION_DEBUG: false
TEMP: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search-arnold/temp.kgtk-search-arnold
KGTK_GRAPH_CACHE: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search-arnold/temp.kgtk-search-arnold/wikidata.sqlite3.db
kgtk: kgtk
KGTK_LABEL_FILE: /Users/amandeep/Github/kgtk-notebooks/datasets/arnold/labels.en.tsv.gz
kypher: kgtk query --graph-cache /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search-arnold/temp.kgtk-search-arnold/wikidata.sqlite3.db
OUT: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search-arnold
label: /Users/amandeep/Github/kgtk-notebooks/datasets/arnold/labels.en.tsv.gz
pagerank_undirected: /Users/amandeep/Github/kgtk-notebooks/datasets/arnold/metadata.pagerank.undirected.tsv.gz
alias: /Users/amandeep/Github/kgtk-no

## Load the files into cache

In [8]:
if create_sqlite_cache:
    ck.load_files_into_cache()

kgtk query --graph-cache /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search-arnold/temp.kgtk-search-arnold/wikidata.sqlite3.db -i "/Users/amandeep/Github/kgtk-notebooks/datasets/arnold/labels.en.tsv.gz" --as label  -i "/Users/amandeep/Github/kgtk-notebooks/datasets/arnold/metadata.pagerank.undirected.tsv.gz" --as pagerank_undirected  -i "/Users/amandeep/Github/kgtk-notebooks/datasets/arnold/aliases.en.tsv.gz" --as alias  -i "/Users/amandeep/Github/kgtk-notebooks/datasets/arnold/descriptions.en.tsv.gz" --as description  -i "/Users/amandeep/Github/kgtk-notebooks/datasets/arnold/claims.tsv.gz" --as claims  -i "/Users/amandeep/Github/kgtk-notebooks/datasets/arnold/metadata.property.datatypes.tsv.gz" --as datatypes  -i "/Users/amandeep/Github/kgtk-notebooks/datasets/arnold/qualifiers.tsv.gz" --as qualifiers  --limit 3
node1	label	node2	id
P10	label	'video'@en	P10-label-en
P1000	label	'record held'@en	P1000-label-en
P1001	label	'applies to jurisdiction'@en	P1001-label-en


## Define the Kypher API

In [9]:
_kapi2 = kapi.KypherApi(graphcache=os.environ['STORE'], loglevel=1, index='auto',
                      maxresults=100, maxcache=0)

## Create a file with `label`, `undirected_pagerank` and `description`

In [12]:
if create_sqlite_cache:
    !kgtk query --gc $STORE \
        -i label pagerank_undirected description\
        --match 'label: (qnode)-[l]->(y), pagerank: (qnode)-[:Pundirected_pagerank]->(pr)' \
        --opt 'description: (qnode)-[:description]->(d)' \
        --return 'qnode as node1, l.label as label, y as node2, upper(y) as `node2;upper`, pr as `node1;pagerank`, ifnull(d, "") as `node1;description`' \
        --order-by 'qnode' \
        -o $OUT/label_pagerank_undirected_description.tsv.gz

### Load this file into cache as well

In [13]:
if create_sqlite_cache:
    !kgtk query --gc $STORE -i $OUT/label_pagerank_undirected_description.tsv.gz --as l_d_pgr_ud --limit 10

node1	label	node2	node2;upper	node1;pagerank	node1;description
P10	label	'video'@en	'VIDEO'@EN	5.200871258153172e-06	'relevant video. For images, use the property P18. For film trailers, qualify with \"object has role\" (P3831)=\"trailer\" (Q622550)'@en
P1000	label	'record held'@en	'RECORD HELD'@EN	4.481127621934422e-06	'notable record achieved by a person or entity, include qualifiers for dates held'@en
P1001	label	'applies to jurisdiction'@en	'APPLIES TO JURISDICTION'@EN	1.8972311563210392e-05	'the item (an institution, law, public office ...) or statement belongs to or has power over or applies to the value (a territorial jurisdiction: a country, state, municipality, ...)'@en
P1004	label	'MusicBrainz place ID'@en	'MUSICBRAINZ PLACE ID'@EN	1.132879689825124e-05	'Identifier for a place in the MusicBrainz open music encyclopedia'@en
P1005	label	'Portuguese National Library ID'@en	'PORTUGUESE NATIONAL LIBRARY ID'@EN	5.363115706968723e-06	'identifier for the Biblioteca Nacional de Portug

## Create the required indices

In [14]:
%%time 
if create_sqlite_cache:
    !kgtk --debug query -i l_d_pgr_ud --idx node1 "node2;upper" label text:node2//name=ldpgridx --gc $STORE --limit 5

[2021-12-03 10:56:09 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_8 AS graph_8_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-12-03 10:56:09 sqlstore]: CREATE INDEX "graph_8_node1_idx" ON "graph_8" ("node1")
[2021-12-03 10:56:09 sqlstore]: ANALYZE "graph_8_node1_idx"
[2021-12-03 10:56:09 sqlstore]: CREATE INDEX "graph_8_node2;upper_idx" ON "graph_8" ("node2;upper")
[2021-12-03 10:56:09 sqlstore]: ANALYZE "graph_8_node2;upper_idx"
[2021-12-03 10:56:09 sqlstore]: CREATE INDEX "graph_8_label_idx" ON "graph_8" ("label")
[2021-12-03 10:56:09 sqlstore]: ANALYZE "graph_8_label_idx"
[2021-12-03 10:56:09 sqlstore]: CREATE VIRTUAL TABLE "graph_8_txtidx_ldpgridx" USING FTS5 ("node2", tokenize="trigram", content="graph_8")
[2021-12-03 10:56:09 sqlstore]: INSERT INTO "graph_8_txtidx_ldpgridx" ("node2") SELECT "node2" FROM graph_8
node1	label	node2	node2;upper	node1;pagerank	node1;description
P10	label	'video'@en	

In [15]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i label --idx label --gc $STORE --limit 5

[2021-12-03 10:56:11 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-12-03 10:56:11 sqlstore]: CREATE INDEX "graph_1_label_idx" ON "graph_1" ("label")
[2021-12-03 10:56:11 sqlstore]: ANALYZE "graph_1_label_idx"
node1	label	node2	id
P10	label	'video'@en	P10-label-en
P1000	label	'record held'@en	P1000-label-en
P1001	label	'applies to jurisdiction'@en	P1001-label-en
P1003	label	'National Library of Romania ID'@en	P1003-label-en
P1004	label	'MusicBrainz place ID'@en	P1004-label-en
CPU times: user 19.5 ms, sys: 15.8 ms, total: 35.3 ms
Wall time: 1.51 s


In [16]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i alias --idx label --gc $STORE --limit 5

[2021-12-03 10:56:12 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_3 AS graph_3_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-12-03 10:56:12 sqlstore]: CREATE INDEX "graph_3_label_idx" ON "graph_3" ("label")
[2021-12-03 10:56:12 sqlstore]: ANALYZE "graph_3_label_idx"
node1	label	node2	id
P10	alias	'gif'@en	P10-alias-en-282226-0
P10	alias	'animation'@en	P10-alias-en-2f86d8-0
P10	alias	'media'@en	P10-alias-en-c1427e-0
P10	alias	'trailer (Commons)'@en	P10-alias-en-c61ab1-0
P1001	alias	'belongs to jurisdiction'@en	P1001-alias-en-0dd7ce-0
CPU times: user 19.3 ms, sys: 16.3 ms, total: 35.6 ms
Wall time: 1.4 s


In [17]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i description --idx id --gc $STORE --limit 5

[2021-12-03 10:56:14 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_4 AS graph_4_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-12-03 10:56:14 sqlstore]: CREATE INDEX "graph_4_id_idx" ON "graph_4" ("id")
[2021-12-03 10:56:14 sqlstore]: ANALYZE "graph_4_id_idx"
node1	label	node2	id
P10	description	'relevant video. For images, use the property P18. For film trailers, qualify with \"object has role\" (P3831)=\"trailer\" (Q622550)'@en	P10-description-en
P1000	description	'notable record achieved by a person or entity, include qualifiers for dates held'@en	P1000-description-en
P1001	description	'the item (an institution, law, public office ...) or statement belongs to or has power over or applies to the value (a territorial jurisdiction: a country, state, municipality, ...)'@en	P1001-description-en
P1003	description	'identifier for authority control used at the National Library of Romania'@en	P1003-descript

In [18]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i claims --idx label node1 node2 id --gc $STORE --limit 5

[2021-12-03 10:56:15 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_5 AS graph_5_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-12-03 10:56:15 sqlstore]: CREATE INDEX "graph_5_label_idx" ON "graph_5" ("label")
[2021-12-03 10:56:16 sqlstore]: ANALYZE "graph_5_label_idx"
[2021-12-03 10:56:16 sqlstore]: CREATE INDEX "graph_5_node1_idx" ON "graph_5" ("node1")
[2021-12-03 10:56:16 sqlstore]: ANALYZE "graph_5_node1_idx"
[2021-12-03 10:56:16 sqlstore]: CREATE INDEX "graph_5_node2_idx" ON "graph_5" ("node2")
[2021-12-03 10:56:16 sqlstore]: ANALYZE "graph_5_node2_idx"
[2021-12-03 10:56:16 sqlstore]: CREATE INDEX "graph_5_id_idx" ON "graph_5" ("id")
[2021-12-03 10:56:16 sqlstore]: ANALYZE "graph_5_id_idx"
node1	label	node2	id	node2;wikidatatype
P10	P31	Q18610173	P10-P31-Q18610173-85ef4d24-0	wikibase-item
P1000	P31	Q18608871	P1000-P31-Q18608871-093affb5-0	wikibase-item
P1001	P1855	Q11696	P1001-P1855-Q11696-cdbf39

In [19]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i datatypes --idx label node1 --gc $STORE --limit 5

[2021-12-03 10:56:18 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_6 AS graph_6_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-12-03 10:56:18 sqlstore]: CREATE INDEX "graph_6_label_idx" ON "graph_6" ("label")
[2021-12-03 10:56:18 sqlstore]: ANALYZE "graph_6_label_idx"
[2021-12-03 10:56:18 sqlstore]: CREATE INDEX "graph_6_node1_idx" ON "graph_6" ("node1")
[2021-12-03 10:56:18 sqlstore]: ANALYZE "graph_6_node1_idx"
node1	label	node2	id
Pisa	label	'is a'@en	Pisa-label-e79b73
Pisa	alias	'isa'@en	Pisa-alias-7773c5
Pisa	description	'Instance or subclass relationship'@en	Pisa-description-0b5cdc
Pisa	P31	Q18616576	Pisa-P31-Q18616576
Pisa	P31	Q28326461	Pisa-P31-Q28326461
CPU times: user 19 ms, sys: 14.8 ms, total: 33.7 ms
Wall time: 1.46 s


In [20]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i qualifiers --idx node2 node1 label --gc $STORE --limit 5

[2021-12-03 10:56:19 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_7 AS graph_7_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2021-12-03 10:56:19 sqlstore]: CREATE INDEX "graph_7_node2_idx" ON "graph_7" ("node2")
[2021-12-03 10:56:20 sqlstore]: ANALYZE "graph_7_node2_idx"
[2021-12-03 10:56:20 sqlstore]: CREATE INDEX "graph_7_node1_idx" ON "graph_7" ("node1")
[2021-12-03 10:56:20 sqlstore]: ANALYZE "graph_7_node1_idx"
[2021-12-03 10:56:20 sqlstore]: CREATE INDEX "graph_7_label_idx" ON "graph_7" ("label")
[2021-12-03 10:56:20 sqlstore]: ANALYZE "graph_7_label_idx"
node1	label	node2	id
P1001-P1855-Q11696-cdbf391b-0	P1001	Q30	P1001-P1855-Q11696-cdbf391b-0-P1001-Q30-0
P1001-P1855-Q12371988-12c10bc0-0	P1001	Q90	P1001-P1855-Q12371988-12c10bc0-0-P1001-Q90-0
P1001-P1855-Q181574-7f428c9b-0	P1001	Q16	P1001-P1855-Q181574-7f428c9b-0-P1001-Q16-0
P1001-P1855-Q181574-7f428c9b-0	P1001	Q30	P1001-P1855-Q181574-7f428c9b-0-P1

## Take a look at cache file content

In [21]:
!kgtk query --gc $STORE --show-cache 

Graph Cache:
DB file: /Volumes/saggu-ssd/wikidata-dwd-v2/kgtk-search-arnold/temp.kgtk-search-arnold/wikidata.sqlite3.db
  size:  175.40 MB   	free:  0 Bytes   	modified:  2021-12-03 10:56:20

KGTK File Information:
alias:
  size:  1.29 MB   	modified:  2021-11-16 14:41:39   	graph:  graph_3
claims:
  size:  6.05 MB   	modified:  2021-12-03 10:55:11   	graph:  graph_5
datatypes:
  size:  22.58 KB   	modified:  2021-11-16 14:41:39   	graph:  graph_6
description:
  size:  1.29 MB   	modified:  2021-11-16 14:41:39   	graph:  graph_4
l_d_pgr_ud:
  size:  3.03 MB   	modified:  2021-12-03 10:56:02   	graph:  graph_8
label:
  size:  1.02 MB   	modified:  2021-11-16 14:41:39   	graph:  graph_1
pagerank_undirected:
  size:  2.11 MB   	modified:  2021-11-16 14:41:39   	graph:  graph_2
qualifiers:
  size:  5.96 MB   	modified:  2021-11-16 14:41:39   	graph:  graph_7

Graph Table Information:
graph_1:
  size:  5.98 MB   	created:  2021-12-03 10:54:35
  header:  ['node1', 'label', 'node2', 'id']
gra

## Define a function to do a `textmatch` search

In [22]:
def text_search_labels(search_text, limit=20):
    text_search_labels_query = _kapi2.get_query(
        doc="Doc string here",
        name=f"text_search_labels_{search_text}",
        inputs='l_d_pgr_ud',
        match='l_d_pgr_ud: (qnode)-[l:label]->(y)',
        where=f'textmatch(y, "{search_text}")',
       ret='distinct qnode as node1, y as label, 10*matchscore(y) as score, cast(l.`node1;pagerank`, float) as prank, l.`node1;description` as description',
       order='score*prank',
       limit=limit
    )
    results =  list([list(x) for x in text_search_labels_query.execute()])
    df = pd.DataFrame(results, columns=['node1', 'label', 'score', 'pagerank', 'description'])
    print(len(df))
    return df
    

In [24]:
text_search_labels('arn sch')

[2021-12-03 10:56:35 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c1."node1" "_aLias.node1", graph_8_c1."node2" "_aLias.label", (? * BM25(txtidx_1.graph_8_txtidx_ldpgridx)) "_aLias.score", CAST(graph_8_c1."node1;pagerank" AS float) "_aLias.prank", graph_8_c1."node1;description" "_aLias.description"
     FROM graph_8 AS graph_8_c1, graph_8_txtidx_ldpgridx AS txtidx_1
     WHERE graph_8_c1."label" = ?
        AND txtidx_1."node2" MATCH ? and txtidx_1.rowid = graph_8_c1.rowid
     ORDER BY ("_aLias.score" * "_aLias.prank") ASC
     LIMIT ?
  PARAS: [10, 'label', 'arn sch', 20]
---------------------------------------------


5


,node1,label,score,pagerank,description
0,Q2685,'Arnold Schwarzenegger'@en,-96.680749,0.000086,"'Austrian-American actor, businessman, bodybui..."
1,Q1732802,'Karl Scharnagl'@en,-110.602797,0.000016,'German politician (1881-1963)'@en
2,Q103575,'Arnold Schaefer'@en,-108.373398,0.000016,'German historian'@en
3,Q432443,'Jan Arnoldus Schouten'@en,-96.680749,0.000011,'Dutch politician (1883-1971)'@en
4,Q80030407,'Karl-Scharnagl-Ring'@en,-100.287496,0.000007,


## Define a function to search for Qnodes Exactly

In [23]:
def exact_search_items(search_text, limit=20):
    search_text = search_text.upper()
    text_search_labels_query =  _kapi2.get_query(
    doc="""
    Create the Kypher query used by 'BrowserBackend.get_node_labels()'
    for case_independent searches.
    Given parameters 'NODE' and 'LANG' retrieve labels for 'NODE' in
    the specified language (using 'any' for 'LANG' retrieves all labels).
    Return distinct 'node1', 'node_label' pairs as the result (we include
    'NODE' as an output to make it easier to union result frames).
    """,
    name=f'exact_search_items{search_text}',
    inputs='l_d_pgr_ud',
    match='l_d_pgr_ud: (n)-[r:label]->(l)',
    where=f'n="{search_text}"',
    ret='distinct n as node1, l as node_label, r.`node1;description` as description',
)
    results =  list([list(x) for x in text_search_labels_query.execute()])
    df = pd.DataFrame(results, columns=['node1', 'label', 'description'])
    print(len(df))
    return df
    

In [24]:
%%time
exact_search_items('q30')

[2021-12-02 22:27:54 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c1."node1" "_aLias.node1", graph_8_c1."node2" "_aLias.node_label", graph_8_c1."node1;description" "_aLias.description"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label" = ?
        AND (graph_8_c1."node1" = ?)
     LIMIT ?
  PARAS: ['label', 'Q30', 100]
---------------------------------------------


1
CPU times: user 67.9 ms, sys: 98.3 ms, total: 166 ms
Wall time: 248 ms


,node1,label,description
0,Q30,'United States of America'@en,'sovereign state in North America'@en


In [25]:
%%time
exact_search_items('Q140')

[2021-12-02 22:27:55 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c1."node1" "_aLias.node1", graph_8_c1."node2" "_aLias.node_label", graph_8_c1."node1;description" "_aLias.description"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label" = ?
        AND (graph_8_c1."node1" = ?)
     LIMIT ?
  PARAS: ['label', 'Q140', 100]
---------------------------------------------


1
CPU times: user 17.3 ms, sys: 4.09 ms, total: 21.4 ms
Wall time: 20.8 ms


,node1,label,description
0,Q140,'lion'@en,'species of big cat'@en


## Define a function to search labels Exactly

In [26]:
def exact_search_labels(search_text, limit=20):
    search_text = f"'{search_text.upper()}'@EN"
    text_search_labels_query =  _kapi2.get_query(
    doc="""
     Exact Match case insensitive query
    """,
    name=f'exact_search_labels{search_text}',
    inputs='l_d_pgr_ud',
    match=f'l_d_pgr_ud: (n)-[r:label]->(l)',
    where=f'r.`node2;upper`="{search_text}"',
    ret='distinct n as node1, l as node_label, cast("-1.0", float) as score, cast(r.`node1;pagerank`, float) as prank, r.`node1;description` as description',
    order='score*prank',
    limit=limit
)
    results =  list([list(x) for x in text_search_labels_query.execute()])
    df = pd.DataFrame(results, columns=['node1', 'label', 'score', 'prank', 'description'])
    print(len(df))
    return df

In [27]:
exact_search_labels('canada')

[2021-12-02 22:27:57 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c1."node1" "_aLias.node1", graph_8_c1."node2" "_aLias.node_label", CAST(? AS float) "_aLias.score", CAST(graph_8_c1."node1;pagerank" AS float) "_aLias.prank", graph_8_c1."node1;description" "_aLias.description"
     FROM graph_8 AS graph_8_c1
     WHERE graph_8_c1."label" = ?
        AND (graph_8_c1."node2;upper" = ?)
     ORDER BY ("_aLias.score" * "_aLias.prank") ASC
     LIMIT ?
  PARAS: ['-1.0', 'label', "'CANADA'@EN", 20]
---------------------------------------------


20


,node1,label,score,prank,description
0,Q16,'Canada'@en,-1.0,1.097894e-03,'sovereign state in North America'@en
1,Q2569593,'Canada'@en,-1.0,8.178669e-08,'former French colony in New France between th...
2,Q13265725,'Canada'@en,-1.0,3.126547e-08,'family name'@en
3,Q13265795,'Canada'@en,-1.0,1.498003e-08,'2012 novel by American author Richard Ford'@en
4,Q2608363,'Canada'@en,-1.0,1.364988e-08,"'village in Appelscha, in the Netherlands'@en"
5,Q18612153,'Canada'@en,-1.0,1.364276e-08,'180th strip of the webcomic xkcd'@en
6,Q103921530,'Canada'@en,-1.0,1.331540e-08,"'Shipwreck off the Scottish Coast, imported fr..."
7,Q99292858,'Canada'@en,-1.0,1.269767e-08,'the country of Canada as depicted in Star Tre...
8,Q14624136,'Canada'@en,-1.0,1.255841e-08,'moth genus of Pteromalidae'@en
9,Q5029265,'Canada'@en,-1.0,1.234508e-08,"'unincorporated community in Kansas, United St..."


## Define a function to fo a `textlike` search

In [28]:
def text_like_search_labels(search_text, limit=20):
    search_label = f"%{'%'.join(search_text.split(' '))}%"
    print(search_text)
    text_search_labels_query = _kapi2.get_query(
        doc="Doc string here",
        name=f"text_like_search_labels_{search_text}",
        inputs='l_d_pgr_ud',
        match='l_d_pgr_ud: (qnode)-[l:label]->(y)',
        where=f'textlike(y, "{search_label}")',
       ret='distinct qnode as node1, y as label, matchscore(y) as score, cast(l.`node1;pagerank`, float) as prank, l.`node1;description` as description',
       order='score*prank',
       limit=limit
    )
    results =  list([list(x) for x in text_search_labels_query.execute()])
    df = pd.DataFrame(results, columns=['node1', 'label', 'score', 'pagerank', 'description'])
    print(len(df))
    return df
    

In [29]:
text_like_search_labels("fifa group b")

[2021-12-02 22:28:01 query]: SQL Translation:
---------------------------------------------
  SELECT DISTINCT graph_8_c1."node1" "_aLias.node1", graph_8_c1."node2" "_aLias.label", BM25(txtidx_1.graph_8_txtidx_ldpgridx) "_aLias.score", CAST(graph_8_c1."node1;pagerank" AS float) "_aLias.prank", graph_8_c1."node1;description" "_aLias.description"
     FROM graph_8 AS graph_8_c1, graph_8_txtidx_ldpgridx AS txtidx_1
     WHERE graph_8_c1."label" = ?
        AND txtidx_1."node2" LIKE ? and txtidx_1.rowid = graph_8_c1.rowid
     ORDER BY ("_aLias.score" * "_aLias.prank") ASC
     LIMIT ?
  PARAS: ['label', '%fifa%group%b%', 20]
---------------------------------------------


fifa group b
20


,node1,label,score,pagerank,description
0,Q31189406,'2018 FIFA World Cup Group B'@en,-14.896818,4.302684e-08,
1,Q10260332,'2014 FIFA World Cup Group B'@en,-14.896818,2.856620e-08,
2,Q17115977,'Category:2014 FIFA World Cup group table temp...,-11.105398,2.377329e-08,'Wikimedia category'@en
3,Q18608402,'Category:2015 FIFA Women\'s World Cup group t...,-10.099565,2.085697e-08,'Wikimedia category'@en
4,Q20730977,'2018 FIFA World Cup qualification – UEFA Grou...,-11.356739,1.593900e-08,
5,Q187411,'2010 FIFA World Cup Group B'@en,-14.896818,1.178271e-08,'football tournament'@en
6,Q39134591,'Template:2018 FIFA World Cup Group B table'@en,-12.183998,1.340353e-08,'Wikimedia template'@en
7,Q39134633,'Template:2018 FIFA World Cup Group C table'@en,-12.183998,1.337304e-08,'Wikimedia template'@en
8,Q39395683,'Template:2018 FIFA World Cup Group D table'@en,-12.183998,1.336062e-08,'Wikimedia template'@en
9,Q39134549,'Template:2018 FIFA World Cup Group A table'@en,-12.183998,1.334831e-08,'Wikimedia template'@en


# Create the JSON lines file required for KGTK search Elasticsearch Index

## Concatenate all the input files

In [ ]:
if create_es_index:
    kgtk(""" cat 
        -i "$GRAPH/labels.en.tsv.gz"
        -i "$GRAPH/aliases.en.tsv.gz"
        -i "$GRAPH/descriptions.en.tsv.gz"
        -i "$GRAPH/claims.tsv.gz"
        -i "$GRAPH/qualifiers.tsv.gz"
        -i "$GRAPH/metadata.pagerank.undirected.tsv.gz"
        -i "$GRAPH/derived.isastar.tsv.gz"
        -o "$OUT"/kgtk.search.unsorted.tsv.gz""")

## Sort

In [ ]:
if create_es_index:
    kgtk(f"""sort --column node1  
        -X "--parallel 8 --buffer-size 60% --temporary-directory {os.environ['TEMP']}" 
        -o "$OUT/kgtk.search.sorted.tsv.gz" """)

## Run `kgtk build-kgtk-search-input` command to create the json lines file

In [ ]:
if create_es_index:
    kgtk("""build-kgtk-search-input --input-file "$OUT/kgtk.search.sorted.tsv.gz"
            --output-file "$OUT"/kgtk.search.sorted.jl 
            --label-properties label 
            --alias-properties alias 
            --extra-alias-properties P1448,P1705,P1477,P1810,P742,P1449 
            --description-properties description 
            --pagerank-properties Pundirected_pagerank 
            --mapping-file "$OUT"/wikidata_es_kgtk_search_mapping.json 
            --property-datatype-file "$GRAPH"/metadata.property.datatypes.tsv.gz""")

## Create the Elasticsearch Index

In [ ]:
if create_es_index:
    os.environ['ESURL'] = f"{es_host}:{es_port}"
    os.environ['ESINDEX'] = es_index

In [ ]:
if create_es_index:
    !curl -H "Content-Type: application/json" -XPUT $ESURL/$ESINDEX -d @"$OUT/wikidata_es_kgtk_search_mapping.json"

## Load the json lines file to the ES Index

The following command requires [table-linker](https://github.com/usc-isi-i2/table-linker) to be installed in the current virtual environment.

In [ ]:
if create_es_index:
    !tl load-elasticsearch-index --es-url $ESURL --es-index $ESINDEX --es-version 7 --kgtk-jl-path "$OUT"/kgtk.search.sorted.jl